In [27]:
import pandas as pd
import numpy as np
import eia
import requests
import json
import yaml
from prophet import Prophet

# Definitions
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
pd.set_option('display.max_columns', 100)    #Display upto 100 columns 
pd.set_option('display.max_rows', 100) 
%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')   

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# To be able to pull modules from src folder 
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

In [4]:
from d00_utils.const import *
from d00_utils.utils import setup_env_vars, load_yml, get_filepath

setup_env_vars()
os.environ.get("EIA_ACCESS_KEY")

2022-02-28 18:18:30,853 - Finding .env file to load entries as environment variables...
2022-02-28 18:18:30,856 - Finished loading environment variables.


'xPWAeqwYEwqmxA0hFnjsCUbcyxEG95ajpq4aX89p'

## Pulling Raw Data (JSON) and Creating Intermediate CSVs
So far, we've pulled all the raw data we're interested in and created intermediate CSVS for it - including handling of failed API requests. 

In [5]:
eia_api_ids = load_yml(EIA_API_IDS_YML_FILEPATH)
for data_type, api_ids_dict in eia_api_ids.items(): 
    print(data_type, api_ids_dict)
    print('\n')

Net_Gen_By_Fuel_MWh {'all_fuels': 'ELEC.GEN.ALL-{}-99.Q', 'coal': 'ELEC.GEN.COW-{}-99.Q', 'natural_gas': 'ELEC.GEN.NG-{}-99.Q', 'nuclear': 'ELEC.GEN.NUC-{}-99.Q', 'hydro': 'ELEC.GEN.HYC-{}-99.Q', 'other_renewables': 'ELEC.GEN.AOR-{}-99.Q', 'wind': 'ELEC.GEN.WND-{}-99.Q', 'solar_utility': 'ELEC.GEN.SUN-{}-99.Q', 'solar_all': 'ELEC.GEN.TSN-{}-99.Q', 'other': 'ELEC.GEN.OTH-{}-99.Q'}


Fuel_Consumption_BTU {'coal': 'ELEC.CONS_TOT_BTU.COW-{}-99.Q', 'natural_gas': 'ELEC.CONS_TOT_BTU.NG-{}-99.Q'}




In [64]:
# Working code so far
from d01_data.get_raw_data import load_all_data
load_all_data(eia_api_ids)

2022-02-28 19:57:05,293 - Invalid response for EIA Series ID: elec.gen.wnd-al-99.q with error message: invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/
2022-02-28 19:57:08,083 - Invalid response for EIA Series ID: elec.gen.wnd-ar-99.q with error message: invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/


## Data Preprocessing 
Next, we need to perform the following preprocessing steps: 
- Handle missing data for all csvs and impute values of zero 
- For csvs missing dates between 2001 and 2021, add missing dates with value of zero
- Perform needed feature engineering using the data so far and save datasets as input to models downstream

In [69]:
# Sample intermediate files for testing: 
impute_null_vals_case = '../data/02_intermediate/Net_Gen_By_Fuel_MWh/AL/Net_Gen_By_Fuel_MWh-solar_all.csv'
missing_dates_case = '../data/02_intermediate/Net_Gen_By_Fuel_MWh/AL/Net_Gen_By_Fuel_MWh-solar_utility.csv'

df_impute = pd.read_csv(impute_null_vals_case)
df_dates = pd.read_csv(missing_dates_case)
df_dates.head()

,date,Net_Gen_By_Fuel_MWh
0,2021-12-31,140.160
1,2021-09-30,156.462
2,2021-06-30,126.982
3,2021-03-31,71.770
4,2020-12-31,71.988


In [70]:
# Null Impute 
df_impute['Net_Gen_By_Fuel_MWh'].fillna(0, inplace=True)
df_impute.head()

,date,Net_Gen_By_Fuel_MWh
0,2021-12-31,0.000
1,2021-09-30,0.000
2,2021-06-30,0.000
3,2021-03-31,0.000
4,2020-12-31,0.000


In [71]:
# Convert to pd.Series prior to imputing missing dates 
print(df_dates.shape)
pd_series = df_dates.set_index('date')['Net_Gen_By_Fuel_MWh']

start_date = "2001-01-01"
end_date = "2021-12-31"
dt_range = pd.date_range(start_date, end_date, freq='Q')
pd_series.index = pd.DatetimeIndex(pd_series.index)

# Impute missing quarters with value of 0
pd_series = pd_series.reindex(dt_range, fill_value=0)

# Create dataframe from pd.Series
df_dates = pd.DataFrame({'date':pd_series.index, 'Net_Gen_By_Fuel_MWh':pd_series.values})
df_dates.shape

(22, 2)


(84, 2)

In [73]:
# Test function after adding to script 
from src.d02_intermediate.clean_raw_data import clean_all_data
clean_all_data(eia_api_ids)

2022-02-28 20:27:41,681 - Invalid response for EIA Series ID: elec.gen.wnd-al-99.q with error message: invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/
2022-02-28 20:27:41,800 - Invalid response for EIA Series ID: elec.gen.wnd-ar-99.q with error message: invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/


## Feature Engineering
With all the data cleaned and imputed, we can move to the next stage of combining the cleaned raw data to create the processed data which will be fed into the ML models for training.